In [85]:
from itertools import *
from copy import copy, deepcopy
from heapq import *

Let $\mathcal{A}$ be our alphabet:

In [62]:
def make_alphabet_entry(i):
    alpha = i + ord('a')
    return chr(alpha),i

alphabet = dict(map(make_alphabet_entry, range(26)))
alphabet.update({' ':26})
n = len(alphabet)
alphabet, n

({' ': 26,
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7,
  'i': 8,
  'j': 9,
  'k': 10,
  'l': 11,
  'm': 12,
  'n': 13,
  'o': 14,
  'p': 15,
  'q': 16,
  'r': 17,
  's': 18,
  't': 19,
  'u': 20,
  'v': 21,
  'w': 22,
  'x': 23,
  'y': 24,
  'z': 25},
 27)

We define a function `encode` that consumes a string and produces a list of integer in our field:

In [3]:
def encode(s): return list(map(lambda i: alphabet[i], s))

inverse_alphabet = {v:k for k,v in alphabet.items()}
def decode(e): return "".join(map(lambda i: inverse_alphabet[i], e))

In [63]:
plain_text = '''A path from a point approximately 330 metres east of the most south
westerly corner of 17 Batherton Close, Widnes and approximately 208
metres east-south-east of the most southerly corner of Unit 3 Foundry
Industrial Estate, Victoria Street, Widnes, proceeding in a generally
east-north-easterly direction for approximately 28 metres to a point
approximately 202 metres east-south-east of the most south-easterly
corner of Unit 4 Foundry Industrial Estate, Victoria Street, and
approximately 347 metres east of the most south-easterly corner of 17
Batherton Close, then proceeding in a generally northerly direction
for approximately 21 metres to a point approximately 210 metres east
of the most south-easterly corner of Unit 5 Foundry Industrial Estate,
Victoria Street, and approximately 202 metres east-south-east of the
most north-easterly corner of Unit 4 Foundry Industrial Estate,
Victoria Street, then proceeding in a generally east-north-east
direction for approximately 64 metres to a point approximately 282
metres east-south-east of the most easterly corner of Unit 2 Foundry
Industrial Estate, Victoria Street, Widnes and approximately 259
metres east of the most southerly corner of Unit 4 Foundry Industrial
Estate, Victoria Street, then proceeding in a generally
east-north-east direction for approximately 350 metres to a point
approximately 3 metres west-north-west of the most north westerly
corner of the boundary fence of the scrap metal yard on the south side
of Cornubia Road, Widnes, and approximately 47 metres west-south-west
of the stub end of Cornubia Road be diverted to a 3 metre wide path
from a point approximately 183 metres east-south-east of the most
easterly corner of Unit 5 Foundry Industrial Estate, Victoria Street
and approximately 272 metres east of the most north-easterly corner of
26 Ann Street West, Widnes, then proceeding in a generally north
easterly direction for approximately 58 metres to a point
approximately 216 metres east-south-east of the most easterly corner
of Unit 4 Foundry Industrial Estate, Victoria Street and approximately
221 metres east of the most southerly corner of Unit 5 Foundry
Industrial Estate, Victoria Street, then proceeding in a generally
easterly direction for approximately 45 metres to a point
approximately 265 metres east-south-east of the most north-easterly
corner of Unit 3 Foundry Industrial Estate, Victoria Street and
approximately 265 metres east of the most southerly corner of Unit 5
Foundry Industrial Estate, Victoria Street, then proceeding in a
generally east-south-east direction for approximately 102 metres to a
point approximately 366 metres east-south-east of the most easterly
corner of Unit 3 Foundry Industrial Estate, Victoria Street and
approximately 463 metres east of the most north easterly corner of 22
Ann Street West, Widnes, then proceeding in a generally
north-north-easterly direction for approximately 19 metres to a point
approximately 368 metres east-south-east of the most easterly corner
of Unit 3 Foundry Industrial Estate, Victoria Street and approximately
512 metres east of the most south easterly corner of 17 Batherton
Close, Widnes then proceeding in a generally east-south, easterly
direction for approximately 16 metres to a point approximately 420
metres east-south-east of the most southerly corner of Unit 2 Foundry
Industrial Estate, Victoria Street and approximately 533 metres east
of the most south-easterly corner of 17 Batherton Close, then
proceeding in a generally east-north-easterly direction for
approximately 240 metres to a point approximately 606 metres east of
the most northerly corner of Unit 4 Foundry Industrial Estate,
Victoria Street and approximately 23 metres south of the most south
westerly corner of the boundary fencing of the scrap metal yard on the
south side of Cornubia Road, Widnes, then proceeding in a generally
northern direction for approximately 44 metres to a point
approximately 3 metres west-north-west of the most north westerly
corner of the boundary fence of the scrap metal yard on the south side
of Cornubia Road and approximately 47 metres west-south-west of the
stub end of Cornubia Road.'''.lower().replace(',','').replace('.','').replace('\n','').replace('-','')#.replace(' ','')
plain_text = "".join(filter(lambda c: not c.isdigit(), plain_text))
m = len(plain_text)
plain_text, m

('a path from a point approximately  metres east of the most southwesterly corner of  batherton close widnes and approximately metres eastsoutheast of the most southerly corner of unit  foundryindustrial estate victoria street widnes proceeding in a generallyeastnortheasterly direction for approximately  metres to a pointapproximately  metres eastsoutheast of the most southeasterlycorner of unit  foundry industrial estate victoria street andapproximately  metres east of the most southeasterly corner of batherton close then proceeding in a generally northerly directionfor approximately  metres to a point approximately  metres eastof the most southeasterly corner of unit  foundry industrial estatevictoria street and approximately  metres eastsoutheast of themost northeasterly corner of unit  foundry industrial estatevictoria street then proceeding in a generally eastnortheastdirection for approximately  metres to a point approximately metres eastsoutheast of the most easterly corner of u

In [64]:
assert decode(encode(plain_text)) == plain_text

In [65]:
encoded_plain_text = encode(plain_text)
encoded_plain_text

[0,
 26,
 15,
 0,
 19,
 7,
 26,
 5,
 17,
 14,
 12,
 26,
 0,
 26,
 15,
 14,
 8,
 13,
 19,
 26,
 0,
 15,
 15,
 17,
 14,
 23,
 8,
 12,
 0,
 19,
 4,
 11,
 24,
 26,
 26,
 12,
 4,
 19,
 17,
 4,
 18,
 26,
 4,
 0,
 18,
 19,
 26,
 14,
 5,
 26,
 19,
 7,
 4,
 26,
 12,
 14,
 18,
 19,
 26,
 18,
 14,
 20,
 19,
 7,
 22,
 4,
 18,
 19,
 4,
 17,
 11,
 24,
 26,
 2,
 14,
 17,
 13,
 4,
 17,
 26,
 14,
 5,
 26,
 26,
 1,
 0,
 19,
 7,
 4,
 17,
 19,
 14,
 13,
 26,
 2,
 11,
 14,
 18,
 4,
 26,
 22,
 8,
 3,
 13,
 4,
 18,
 26,
 0,
 13,
 3,
 26,
 0,
 15,
 15,
 17,
 14,
 23,
 8,
 12,
 0,
 19,
 4,
 11,
 24,
 26,
 12,
 4,
 19,
 17,
 4,
 18,
 26,
 4,
 0,
 18,
 19,
 18,
 14,
 20,
 19,
 7,
 4,
 0,
 18,
 19,
 26,
 14,
 5,
 26,
 19,
 7,
 4,
 26,
 12,
 14,
 18,
 19,
 26,
 18,
 14,
 20,
 19,
 7,
 4,
 17,
 11,
 24,
 26,
 2,
 14,
 17,
 13,
 4,
 17,
 26,
 14,
 5,
 26,
 20,
 13,
 8,
 19,
 26,
 26,
 5,
 14,
 20,
 13,
 3,
 17,
 24,
 8,
 13,
 3,
 20,
 18,
 19,
 17,
 8,
 0,
 11,
 26,
 4,
 18,
 19,
 0,
 19,
 4,
 26,
 21,
 8,
 2,
 19,


Let $k \in \left(\frac{\mathbb{Z}}{n\mathbb{Z}}\right)^{m}$ be a *key* of length $m\in\mathbb{N}$, for example:

In [66]:
# it should be better to randomly generate it to keep it safe
k = encode("ericsmullyan")
k, len(k)

([4, 17, 8, 2, 18, 12, 20, 11, 11, 24, 0, 13], 12)

Now we are in the position to define the `encrypt` function:

In [67]:
def encrypt(message, key):
    return [(p+v)%n for p,v in zip(message, cycle(key))]

In [68]:
cipher_text = encrypt(encoded_plain_text, k)
decode(cipher_text)

'eqxckttqblmmeqxq zmklmpdsnqoseywix zijzgjlylcq ajqajwlfzcq eskajnqldpolkdtwteqkkzc mfrajwcmzyxcysimbnuxypp nruhcgakzhfmnxvt ryydbbsmir vj ndsbaexqwhreapkjoexq qleapbiymgezpwctzqxu mjhbx nyooyvrubukcblwxeexragrgbndlrveq viqydktiqrv bgchnpbdvrxhkelukrbnrvrtnpqucdkodxymcjeybwv qmhmekuhykcoddrxri qtxytrpohbdqmbpp fsqibg bydypbveekdmmpwv mqvatwdtplpteskajwmldklfmxymbd ldkpogxymcjeybwvcavdmtr zkekifdqnqlzxbixi hk viuuwkbsfejmbmuwdzoindiatwqmklkdntfzqoufldblkdqugkcyckbaexqwhreapkjoexq qleaplptrvbfbu kypo ajqjcktybdlnmgbwuwlmspk bvekgwpbyrxi drhiwzyblilkddwtktybwv qmhmekuhyqlrmefxtfibxlqeybqhowekpcxtadrhrfugdkypbveekdmmpwv mqvatwdtplptajqajwlfzcq eskajwmldpolkdtwteqkkzc grzabrrheyarkdzvfldmbtylmiiackqotnqodmrhukcypdxa hqirgchhtjafibfbryydbbsmir vj ndsbaexqwhreapxlsfddwtktylcqedpohefcgpbxosdkvkkltqzrnqvohkepncdoinpqmukmmpffcfshqcrdmbpbtmxymprakznbeqmdob ztlkde ihincjtplpt shajwmldofrrgjqqelzzbxabthwz yudpiymdcmviqlkdl ndfwkeetl mraazuckqeikjefvv bwmldcluflviuklhqkqhrdcwuklylcqedpohefcgpbxosdkv

In [69]:
def frequency(elem, lst):
    return lst.count(elem)

def coincidence_index(lst, A=alphabet):
    sums = 0
    freqs = {}
    denom = len(lst)
    #if denom == 0 or denom == 1: return 0,{}
    for k,v in A.items(): freqs[k] = frequency(v, lst)
    ci = sum(v*(v-1) for v in freqs.values())/(denom*(denom-1))
    return ci, freqs

def mutual_coincidence_index(fst, snd, A=alphabet):
    fst_ci = coincidence_index(fst, A)[1]
    snd_ci = coincidence_index(snd, A)[1]
    return sum(fst_ci[k] * snd_ci[k] for k in A.keys())/(len(fst)*len(snd))
    #return sum((fst_ci[k]/m)*(snd_ci[k]/m) for k,v in A.items())

In [70]:
coincidence_index(encoded_plain_text)

(0.07651384358098406,
 {' ': 567,
  'a': 259,
  'b': 15,
  'c': 76,
  'd': 92,
  'e': 440,
  'f': 89,
  'g': 23,
  'h': 99,
  'i': 188,
  'j': 0,
  'k': 0,
  'l': 112,
  'm': 104,
  'n': 201,
  'o': 305,
  'p': 103,
  'q': 0,
  'r': 305,
  's': 232,
  't': 427,
  'u': 85,
  'v': 16,
  'w': 23,
  'x': 37,
  'y': 100,
  'z': 0})

In [71]:
coincidence_index(cipher_text)

(0.040097676798916376,
 {' ': 159,
  'a': 99,
  'b': 188,
  'c': 126,
  'd': 187,
  'e': 191,
  'f': 129,
  'g': 81,
  'h': 134,
  'i': 143,
  'j': 93,
  'k': 238,
  'l': 192,
  'm': 185,
  'n': 84,
  'o': 102,
  'p': 160,
  'q': 235,
  'r': 175,
  's': 83,
  't': 151,
  'u': 112,
  'v': 161,
  'w': 112,
  'x': 124,
  'y': 124,
  'z': 130})

In [72]:
def spread(message, block_length):
    return [message[i:i+block_length] for i in range(0, len(message), block_length)]
    
def col(spreaded, c, joiner=""):
    column = [lst[c] if c < len(lst) else None for lst in spreaded]
    ready = list(filter(lambda i: i is not None, column))
    return joiner.join(ready) if joiner is not None else ready

In [73]:
spreaded = spread(cipher_text, 5)
spreaded

[[4, 16, 23, 2, 10],
 [19, 19, 16, 1, 11],
 [12, 12, 4, 16, 23],
 [16, 26, 25, 12, 10],
 [11, 12, 15, 3, 18],
 [13, 16, 14, 18, 4],
 [24, 22, 8, 23, 26],
 [25, 8, 9, 25, 6],
 [9, 11, 24, 11, 2],
 [16, 26, 0, 9, 16],
 [0, 9, 22, 11, 5],
 [25, 2, 16, 26, 4],
 [18, 10, 0, 9, 13],
 [16, 11, 3, 15, 14],
 [11, 10, 3, 19, 22],
 [19, 4, 16, 10, 10],
 [25, 2, 26, 12, 5],
 [17, 0, 9, 22, 2],
 [12, 25, 24, 23, 2],
 [24, 18, 8, 12, 1],
 [13, 20, 23, 24, 15],
 [15, 26, 13, 17, 20],
 [7, 2, 6, 0, 10],
 [25, 7, 5, 12, 13],
 [23, 21, 19, 26, 17],
 [24, 24, 3, 1, 1],
 [18, 12, 8, 17, 26],
 [21, 9, 26, 13, 3],
 [18, 1, 0, 4, 23],
 [16, 22, 7, 17, 4],
 [0, 15, 10, 9, 14],
 [4, 23, 16, 26, 16],
 [11, 4, 0, 15, 1],
 [8, 24, 12, 6, 4],
 [25, 15, 22, 2, 19],
 [25, 16, 23, 20, 26],
 [12, 9, 7, 1, 23],
 [26, 13, 24, 14, 14],
 [24, 21, 17, 20, 1],
 [20, 10, 2, 1, 11],
 [22, 23, 4, 4, 23],
 [17, 0, 6, 17, 6],
 [1, 13, 3, 11, 17],
 [21, 4, 16, 26, 21],
 [8, 16, 24, 3, 10],
 [19, 8, 16, 17, 21],
 [26, 1, 6, 2, 7],

In [74]:
col(spreaded,2,None)

[23,
 16,
 4,
 25,
 15,
 14,
 8,
 9,
 24,
 0,
 22,
 16,
 0,
 3,
 3,
 16,
 26,
 9,
 24,
 8,
 23,
 13,
 6,
 5,
 19,
 3,
 8,
 26,
 0,
 7,
 10,
 16,
 0,
 12,
 22,
 23,
 7,
 24,
 17,
 2,
 4,
 6,
 3,
 16,
 24,
 16,
 6,
 1,
 7,
 10,
 21,
 16,
 14,
 2,
 22,
 7,
 7,
 3,
 8,
 24,
 7,
 1,
 18,
 26,
 15,
 10,
 22,
 21,
 19,
 4,
 22,
 11,
 12,
 3,
 23,
 4,
 2,
 19,
 4,
 16,
 23,
 26,
 8,
 1,
 12,
 3,
 3,
 16,
 3,
 16,
 3,
 16,
 24,
 4,
 17,
 9,
 26,
 15,
 21,
 26,
 26,
 2,
 3,
 1,
 12,
 1,
 22,
 23,
 7,
 1,
 3,
 19,
 26,
 4,
 16,
 5,
 1,
 24,
 22,
 23,
 7,
 3,
 21,
 12,
 26,
 19,
 11,
 16,
 5,
 4,
 22,
 14,
 22,
 10,
 17,
 17,
 17,
 5,
 19,
 8,
 14,
 3,
 10,
 23,
 8,
 7,
 8,
 24,
 18,
 21,
 18,
 16,
 0,
 5,
 10,
 16,
 7,
 15,
 3,
 11,
 13,
 10,
 3,
 16,
 12,
 5,
 17,
 1,
 12,
 25,
 12,
 25,
 4,
 13,
 11,
 7,
 11,
 17,
 4,
 23,
 22,
 3,
 3,
 16,
 26,
 10,
 26,
 25,
 4,
 5,
 22,
 11,
 8,
 16,
 3,
 11,
 4,
 4,
 1,
 10,
 19,
 16,
 21,
 5,
 26,
 10,
 18,
 3,
 19,
 15,
 24,
 5,
 5,
 10,
 8,
 24,
 13,
 10

In [78]:
def analyze(cipher_text):
    res = {}
    # we discard the case where the key length equals the
    # length of the cipher text, since it is the case of 
    # OneTimePad chiper, which is unbreakable!
    for d in range(2, len(cipher_text)):
        spreaded = spread(cipher_text, d)
        first_col = col(spreaded, 0, joiner=None)
        res[d] = []
        for c in range(d):
            column = col(spreaded, c, joiner=None)
            if len(column) < 2: continue
            mci = coincidence_index(column)[0]
            res[d].append(mci)
    return res

def guess_key_length(analysis):
    candidates = {}
    for k,v in analysis.items():
        cs = list(filter(lambda i: i > .06, v))
        if cs and len(cs) > k/2: candidates[k] = cs
    return candidates

In [76]:
analysis = analyze(cipher_text)

In [77]:
guess_key_length(analysis)

{12: [0.07623931623931623,
  0.08001899335232669,
  0.07367521367521368,
  0.07792972459639126,
  0.07287749287749287,
  0.08797720797720798,
  0.07447293447293447,
  0.08398860398860399,
  0.07504273504273504,
  0.0750997150997151,
  0.06621947024423805,
  0.08196689982035699],
 24: [0.07013557524804968,
  0.07839127471029311,
  0.07604332348708627,
  0.07521017950465803,
  0.07869423615844884,
  0.08982806937817163,
  0.07680072710747557,
  0.0860410512762251,
  0.07581610240096948,
  0.07930015905476029,
  0.07576106126830764,
  0.07652787362932291,
  0.08258569128134345,
  0.07967180430948546,
  0.07062341844950541,
  0.0791350356567748,
  0.06510236945019554,
  0.08526953454489686,
  0.06901311249137336,
  0.08028525419829767,
  0.07039337474120083,
  0.07185031822712983,
  0.08580630319760754],
 36: [0.08375806999660211,
  0.0800203873598369,
  0.07764186204553176,
  0.06846755011892626,
  0.07220523275569148,
  0.08256880733944955,
  0.07560312606184166,
  0.09259259259259259,
 

In [95]:
def find_key(cipher_text, key_length, threshold=.05):
    key = {}
    spreaded = spread(cipher_text, key_length)
    for c in range(key_length):
        column = col(spreaded, c, joiner=None)
        for v in range(0,n):
            shifted = list(map(lambda x: (x-v)%n, column))
            key[c,v]={}
            for a in range(key_length):
                if a == c and v == 0: continue
                another = col(spreaded, a, joiner=None)
                mci = mutual_coincidence_index(shifted, another)
                key[c,v].update({a:mci})
    
    filtered = {}
    for (c, v), cols_mci in key.items():
        if c not in filtered: filtered.update({c:[]})
        for a, mci in cols_mci.items():
            if mci > threshold: heappush(filtered[c], (1-mci,v))
    
    return filtered

In [96]:
find_key(cipher_text, key_length=12)

{0: [(0.918087573964497, 19),
  (0.9198485207100592, 20),
  (0.9244402366863905, 11),
  (0.9213727810650888, 14),
  (0.9211396011396011, 18),
  (0.9268923076923077, 23),
  (0.9262106508875739, 7),
  (0.9287939221272554, 4),
  (0.9258508875739645, 2),
  (0.9234556213017752, 20),
  (0.9493869822485207, 6),
  (0.9498130177514793, 9),
  (0.9488, 14),
  (0.9275266272189349, 13)],
 1: [(0.9178887573964497, 5),
  (0.9194130177514793, 6),
  (0.9217283950617284, 4),
  (0.9213727810650888, 13),
  (0.9476733727810651, 22),
  (0.9448426035502958, 0),
  (0.9257278106508876, 20),
  (0.9255668639053254, 9),
  (0.9224994082840237, 6),
  (0.9491313609467456, 10),
  (0.9497372781065089, 1),
  (0.9485254437869822, 12),
  (0.9461112426035503, 19),
  (0.9296852071005917, 26),
  (0.9294112060778728, 17),
  (0.9495763313609468, 6),
  (0.9485633136094674, 18),
  (0.9266840236686391, 15),
  (0.9236544378698225, 24),
  (0.9498603550295858, 5)],
 2: [(0.9196497041420119, 23),
  (0.9233428300094967, 22),
  (0.921